# 03_compute_bandSNR

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [2]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
band = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# set path
pre_root = '/nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids'
post_root = pjoin(pre_root, 'derivatives', 'preproc_meg-mne_mri-fmriprep')

results_pth = '/nfs/e5/studyforrest/forrest_movie_meg/tech_val_results'
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

In [3]:
def load_megdata(meg_root, sub, run):
    """
    load meg data
    desp: str, ['raw', 'preproced']
    
    """
    sub_path = BIDSPath(subject=sub, run=run, task='movie', session='movie', root=meg_root)
    meg_data = mne_bids.read_raw_bids(sub_path)
    
    # get data of meg sensors
    ch_name_picks = mne.pick_channels_regexp(meg_data.ch_names, regexp='M[LRZ]...-4503')
    type_picks = mne.pick_types(meg_data.info, meg=True)
    picks= np.intersect1d(ch_name_picks, type_picks)
    data = meg_data.get_data(picks=picks)
    
    # get data during movie playing
    event = mne.find_events(meg_data, stim_channel='UPPT001', min_duration=2/meg_data.info['sfreq'])
    sample = event[:, 0]
    data = data.take(sample, axis=1)
    
    return data

In [4]:
def bandSNR(data, fs):
    """
    Calculate bandSNR for 
    Parameters
    ----------
    data: array
        [n_channels, n_samples]
        
    Returns
    -------
    bandSNR : array
        [n_channels, n_bands]
    """
    
    # remove linear trend
    data_detrend = signal.detrend(data, axis=-1)

    # convert to frequency domain        
    freqs, psd = signal.welch(data_detrend, fs=fs)

    band = [[1, 4], [4, 8], [8, 13], [13, 30], [30, 100]]
    # "delta": 1-4Hz
    # "theta": 4-8Hz
    # "alpha": 8-13Hz
    # "beta": 13-30Hz
    # "gamma": 30-100Hz

    bandSNR = [np.sum(psd[:, (freqs>i[0]) * (freqs<i[1])], axis=-1) / np.sum(psd[:, freqs<0.5*fs], axis=-1) for i in band]
    bandSNR = np.asarray(bandSNR).T
    
    return bandSNR

In [ ]:
# compute bandSNR
pre_bandSNR_data = {}
post_bandSNR_data = {}

for sub in sub_list:
    pre_sub = []
    post_sub = []

    # get runlist
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
        
    for run in run_ls:       
        # compute bandSNR for raw meg data
        pre_data = load_megdata(pre_root, sub=sub, run=run)
        pre_bandSNR = bandSNR(pre_data, fs=600)
        pre_sub.append(pre_bandSNR)
        
        # compute bandSNR for preprocessed meg data
        post_data = load_megdata(post_root, sub=sub, run=run)
        post_bandSNR = bandSNR(post_data, fs=600)
        post_sub.append(post_bandSNR)

        print('sub-{0} run-{1} done'.format(sub, run))
    
    pre_bandSNR_data[sub] = pre_sub
    post_bandSNR_data[sub] = post_sub
    

In [7]:
# save bandSNR
for sub in sub_list[1:]:
    pre_bandSNR_data[sub].append(np.nan)
    post_bandSNR_data[sub].append(np.nan)
pre_df = pd.DataFrame(pre_bandSNR_data, columns=sub_list, index=run_list+['09'])
post_df = pd.DataFrame(post_bandSNR_data, columns=sub_list, index=run_list+['09'])
pre_df.to_pickle(pjoin(results_pth, 'bandSNR_pre.pickle'))
post_df.to_pickle(pjoin(results_pth, 'bandSNR_post.pickle'))